In [117]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [118]:
%run ./DataAPIs.ipynb

In [119]:
df = AustinDataAPI().get_spark_df()

2025-01-05 15:12:26,277 - INFO - Request URL: 'https://login.microsoftonline.com/0e42f922-cf76-4f49-8908-8ec2a3408c23/v2.0/.well-known/openid-configuration'
Request method: 'GET'
Request headers:
    'User-Agent': 'azsdk-python-identity/1.19.0 Python/3.12.7 (macOS-10.16-x86_64-i386-64bit)'
No body was attached to the request
2025-01-05 15:12:26,622 - INFO - Response status: 200
Response headers:
    'Cache-Control': 'max-age=86400, private'
    'Content-Type': 'application/json; charset=utf-8'
    'Strict-Transport-Security': 'REDACTED'
    'X-Content-Type-Options': 'REDACTED'
    'Access-Control-Allow-Origin': 'REDACTED'
    'Access-Control-Allow-Methods': 'REDACTED'
    'P3P': 'REDACTED'
    'x-ms-request-id': 'f2b003ca-e2dc-4b13-89fd-fd6077dfaf00'
    'x-ms-ests-server': 'REDACTED'
    'x-ms-srs': 'REDACTED'
    'Content-Security-Policy-Report-Only': 'REDACTED'
    'X-XSS-Protection': 'REDACTED'
    'Set-Cookie': 'REDACTED'
    'Date': 'Sun, 05 Jan 2025 21:12:27 GMT'
    'Content-Le

In [120]:
df.printSchema()

root
 |-- :@computed_region_99rk_ypn4: string (nullable = true)
 |-- :@computed_region_ap3j_c5bq: string (nullable = true)
 |-- :@computed_region_e9j2_6w3z: string (nullable = true)
 |-- :@computed_region_g44y_bfnm: string (nullable = true)
 |-- :@computed_region_jcrc_4uuy: string (nullable = true)
 |-- :@computed_region_m2th_e4b7: string (nullable = true)
 |-- :@computed_region_pgdr_vyqg: string (nullable = true)
 |-- :@computed_region_q9nd_rr82: string (nullable = true)
 |-- :@computed_region_rxpj_nzrk: string (nullable = true)
 |-- :@computed_region_u569_ruue: string (nullable = true)
 |-- address: string (nullable = true)
 |-- agency: string (nullable = true)
 |-- issue_reported: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- location: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- longitude: string (nullable = true)
 |-- published_date: string (nullable = true)
 |-- traffic_report_id: string (nullable = tr

In [121]:
df = df.withColumn("incident_date", to_date("published_date"))

In [122]:
df_with_salt = df.withColumn("salted_key", (col("incident_date") + (rand() * 1000).cast("int")))
df_salted_repartitioned = df_with_salt.repartition("salted_key")

In [123]:
df = df_salted_repartitioned.select(
    'address',
    'agency',
    'issue_reported',
    'location',
    'latitude',
    'longitude',
    'published_date',
    'traffic_report_id',
    'traffic_report_status',
    'traffic_report_status_date_time',
    'incident_date',
    'salted_key'
)

In [124]:
df.limit(5).toPandas()

,address,agency,issue_reported,location,latitude,longitude,published_date,traffic_report_id,traffic_report_status,traffic_report_status_date_time,incident_date,salted_key
0,3663 Bee Cave Rd,TRAVIS COUNTY SHERIF,COLLISION,"{'coordinates': '[-97.806563, 30.279129]', 'ty...",30.279129,-97.806563,2024-12-11T15:51:04.000Z,0C625939CBE82C0DA99CB0B59ACE05648B97290D_17339...,ARCHIVED,2024-12-11T16:25:20.000Z,2024-12-11,2026-10-09
1,10610 FM 969 RD,TRAVIS COUNTY SHERIF,COLLISION WITH INJURY,"{'coordinates': '[-97.620521, 30.261869]', 'ty...",30.261869,-97.620521,2024-12-13T07:04:47.000Z,8F7A4CF1D8FB52D12619FB0C6CAFC42ABF5A5E61_17340...,ARCHIVED,2024-12-13T07:20:18.000Z,2024-12-13,2026-08-18
2,7710 Cameron Rd,AUSTIN PD,Traffic Hazard,"{'coordinates': '[-97.686669, 30.332243]', 'ty...",30.332243,-97.686669,2024-12-13T12:14:00.000Z,CDF61942C1B0FC67819367EDC682E0C5FD5E427F_17340...,ARCHIVED,2024-12-13T13:57:57.000Z,2024-12-13,2025-10-21
3,4715 S LAMAR BLVD NB,TRAVIS COUNTY SHERIF,COLLISION,"{'coordinates': '[-97.81135, 30.232639]', 'typ...",30.232639,-97.81135,2024-12-16T20:30:43.000Z,A6FB51F36210FA55A23ACA9E4C8C3F346506A42A_17343...,ARCHIVED,2024-12-16T21:10:19.000Z,2024-12-16,2025-02-16
4,E Us 290 Hwy Eb / Kimbro Rd,TRAVIS COUNTY SHERIF,TRFC HAZD/ DEBRIS,"{'coordinates': '[-97.500708, 30.350458]', 'ty...",30.350458,-97.500708,2024-12-17T04:04:35.000Z,258635DEFA94EC462FBE0C1AF91C7B59BEF1D1E4_17344...,ARCHIVED,2024-12-17T04:25:19.000Z,2024-12-17,2025-10-21


In [135]:
keys = ["coordinates", "type"]
cleaned_df = df \
    .select("*", *[col("location").getItem(k).alias(k) for k in keys]) \
    .select(
        "traffic_report_id",
        "issue_reported",
        col("longitude").cast("double"),
        col("latitude").cast("double"),
        "type",
        "traffic_report_status",
        "traffic_report_status_date_time"
    )

cleaned_df.show()

+--------------------+--------------------+----------+---------+-----+---------------------+-------------------------------+
|   traffic_report_id|      issue_reported| longitude| latitude| type|traffic_report_status|traffic_report_status_date_time|
+--------------------+--------------------+----------+---------+-----+---------------------+-------------------------------+
|0C625939CBE82C0DA...|           COLLISION|-97.806563|30.279129|Point|             ARCHIVED|           2024-12-11T16:25:...|
|8F7A4CF1D8FB52D12...|COLLISION WITH IN...|-97.620521|30.261869|Point|             ARCHIVED|           2024-12-13T07:20:...|
|CDF61942C1B0FC678...|      Traffic Hazard|-97.686669|30.332243|Point|             ARCHIVED|           2024-12-13T13:57:...|
|A6FB51F36210FA55A...|           COLLISION| -97.81135|30.232639|Point|             ARCHIVED|           2024-12-16T21:10:...|
|258635DEFA94EC462...|   TRFC HAZD/ DEBRIS|-97.500708|30.350458|Point|             ARCHIVED|           2024-12-17T04:25:...|


In [126]:
cleaned_df \
    .groupBy("issue_reported") \
    .count() \
    .orderBy("count", ascending=False) \
    .toPandas()

,issue_reported,count
0,Traffic Hazard,120529
1,Crash Urgent,98235
2,Crash Service,55939
3,COLLISION,41258
4,TRFC HAZD/ DEBRIS,30826
5,LOOSE LIVESTOCK,11068
6,COLLISION WITH INJURY,10593
7,zSTALLED VEHICLE,8938
8,Stalled Vehicle,8590
9,COLLISN/ LVNG SCN,7975


In [137]:
# Clean up the types of traffic incidents
cleaned_df.createOrReplaceTempView("traffic_incident_df")
cleaned_df = spark.sql("""
    SELECT
        traffic_report_id,
        CASE WHEN UPPER(issue_reported) LIKE '%COLLISION%' 
                OR UPPER(issue_reported) LIKE '%COLLISN%' 
                OR UPPER(issue_reported) LIKE '%FLEET ACC%' THEN 'COLLISION'
             WHEN UPPER(issue_reported) LIKE '%HAZD%' 
                OR UPPER(issue_reported) LIKE '%HAZARD%' 
                OR UPPER(issue_reported) LIKE '%HZRD%' 
                OR UPPER(issue_reported) LIKE '%AUTO/ PED%' THEN 'TRAFFIC HAZARD'
             WHEN  UPPER(issue_reported) LIKE '%TRAFFIC IMPEDIMENT%' 
                OR UPPER(issue_reported) LIKE '%BLOCKED DRIV/ HWY%'
                OR UPPER(issue_reported) LIKE '%OBSTRUCT HWY%' THEN 'OBSTRUCTION'
             WHEN UPPER(issue_reported) LIKE '%STALLED%' THEN 'STALLED VEHICLE'
             WHEN UPPER(issue_reported) IN ('ICY ROADWAY', 'HIGH WATER') THEN 'WEATHER'
             ELSE UPPER(issue_reported)
         END AS issue_reported,
         longitude,
         latitude,
         type,
         traffic_report_status,
         traffic_report_status_date_time
    FROM traffic_incident_df
    WHERE issue_reported NOT IN ('TRAFFIC FATALITY', 'BOAT ACCIDENT')
""")

cleaned_df \
    .groupBy("issue_reported") \
    .count() \
    .orderBy("count", ascending=False) \
    .show()

+---------------+------+
| issue_reported| count|
+---------------+------+
| TRAFFIC HAZARD|151540|
|   CRASH URGENT| 98235|
|      COLLISION| 61661|
|  CRASH SERVICE| 55939|
|STALLED VEHICLE| 17528|
|LOOSE LIVESTOCK| 11068|
|    OBSTRUCTION|  7104|
|   VEHICLE FIRE|  1534|
|        WEATHER|    79|
+---------------+------+



In [ ]:
cleaned_df \
    .groupBy("type") \
    .count() \
    .show()

In [ ]:
cleaned_df \
    .filter("type is null") \
    .show()

In [ ]:
cleaned_df \
    .filter("latitude = 0 and longitude = 0") \
    .count()

In [ ]:
cleaned_df \
    .filter("latitude != 0 and longitude != 0") \
    .show()

In [ ]:
cleaned_df \
    .groupBy("traffic_report_status") \
    .count() \
    .orderBy("count") \
    .show()

In [142]:
cleaned_df \
    .select(
        "traffic_report_id",
        "issue_reported",
        "longitude",
        "latitude",
        "traffic_report_status_date_time"
    ) \
    .show()

+--------------------+---------------+----------+---------+-------------------------------+
|   traffic_report_id| issue_reported| longitude| latitude|traffic_report_status_date_time|
+--------------------+---------------+----------+---------+-------------------------------+
|0C625939CBE82C0DA...|      COLLISION|-97.806563|30.279129|           2024-12-11T16:25:...|
|8F7A4CF1D8FB52D12...|      COLLISION|-97.620521|30.261869|           2024-12-13T07:20:...|
|CDF61942C1B0FC678...| TRAFFIC HAZARD|-97.686669|30.332243|           2024-12-13T13:57:...|
|A6FB51F36210FA55A...|      COLLISION| -97.81135|30.232639|           2024-12-16T21:10:...|
|258635DEFA94EC462...| TRAFFIC HAZARD|-97.500708|30.350458|           2024-12-17T04:25:...|
|1EB904B6D0C2A41AF...|   CRASH URGENT| -97.79717|30.468546|           2024-12-18T01:55:...|
|CF39DA87BBEF9B83A...|   CRASH URGENT|-97.739401| 30.37706|           2024-12-18T04:05:...|
|C718AA30A6FF882B7...|   CRASH URGENT|-97.846052|30.400511|           2024-12-18